In [1]:
import argparse
import os
import torch

from exp.exp_informer import Exp_Informer
from data.datautils import *

In [2]:
from attrdict import AttrDict

In [3]:
args = AttrDict({"activation": 'gelu', "attn": 'prob', "batch_size": 8, 
        "c_out": 7, "checkpoints": './checkpoints/', "cols": None, 
        "d_ff": 2048, "d_layers": 1, "d_model": 64, "data": 'ECL', 
        "data_path": 'ECL.csv', "dec_in": 1, "des": 'Exp', "detail_freq": 'h', 
        "devices": '0,1,2,3', "distil": True, "do_predict": False, "dropout": 0.05, 
        "e_layers": 2, "embed": 'timeF', "enc_in": 1, "factor": 3, "features": 'M', "freq": 'h', 
        "gpu": 0, "inverse": False, "itr": 1, "label_len": 48, "learning_rate": 0.001, "loss": 'mse', 
        "lradj": 'type1', "mix": True, "model": 'tcn-moco', "n_heads": 8, "num_workers": 0, 
        "output_attention": False, "padding": 0, "patience": 3, "pred_len": 24, "root_path": './data/', 
        "s_layers": [3, 2, 1], "seq_len": 48, "target": 'OT', "train_epochs": 20, "use_amp": False, "l2norm": True,
        "use_gpu": True, "use_multi_gpu": False, "loss_lambda": 0.5, "mask_rate": 0.2, "kernel_size": 3, 
        "moco_average_pool": False, "data_aug": "cost", "cos_lr": True, "des_path": "./twophase_result/",
        "freeze_encoder": False, "svm_evaluate": True})

In [4]:
args.use_gpu = False#True if torch.cuda.is_available() and args.use_gpu else False

In [5]:
args.use_gpu 

False

In [6]:
if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

In [7]:
data_parser = {
    'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1]},
    'WTH':{'data':'WTH.csv','T':'WetBulbCelsius','M':[12,12,12],'S':[1,1,1],'MS':[12,12,1]},
    'ECL':{'data':'ECL.csv','T':'MT_320','M':[321,321,321],'S':[1,1,1],'MS':[321,321,1]},
    'Solar':{'data':'solar_AL.csv','T':'POWER_136','M':[137,137,137],'S':[1,1,1],'MS':[137,137,1]},
}
if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]

# args.s_layers = [int(s_l) for s_l in args.s_layers.replace(' ','').split(',')]
args.detail_freq = args.freq
args.freq = args.freq[-1:]

print('Args in experiment:')
print(args)

Exp = Exp_Informer

Args in experiment:
AttrDict({'activation': 'gelu', 'attn': 'prob', 'batch_size': 8, 'c_out': 321, 'checkpoints': './checkpoints/', 'cols': None, 'd_ff': 2048, 'd_layers': 1, 'd_model': 64, 'data': 'ECL', 'data_path': 'ECL.csv', 'dec_in': 321, 'des': 'Exp', 'detail_freq': 'h', 'devices': '0,1,2,3', 'distil': True, 'do_predict': False, 'dropout': 0.05, 'e_layers': 2, 'embed': 'timeF', 'enc_in': 321, 'factor': 3, 'features': 'M', 'freq': 'h', 'gpu': 0, 'inverse': False, 'itr': 1, 'label_len': 48, 'learning_rate': 0.001, 'loss': 'mse', 'lradj': 'type1', 'mix': True, 'model': 'tcn-moco', 'n_heads': 8, 'num_workers': 0, 'output_attention': False, 'padding': 0, 'patience': 3, 'pred_len': 24, 'root_path': './data/', 's_layers': [3, 2, 1], 'seq_len': 48, 'target': 'MT_320', 'train_epochs': 20, 'use_amp': False, 'l2norm': True, 'use_gpu': False, 'use_multi_gpu': False, 'loss_lambda': 0.5, 'mask_rate': 0.2, 'kernel_size': 3, 'moco_average_pool': False, 'data_aug': 'cost', 'cos_lr': True, 'des_pa

In [8]:
args.enc_in

321

In [9]:
for ii in range(args.itr):
    # setting record of experiments
    setting = '{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_at{}_fc{}_eb{}_dt{}_mx{}_{}_{}'.format(args.model, args.data, args.features, 
                args.seq_len, args.label_len, args.pred_len,
                args.d_model, args.n_heads, args.e_layers, args.d_layers, args.d_ff, args.attn, args.factor, 
                args.embed, args.distil, args.mix, args.des, ii)

    exp = Exp(args) # set experiments
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
#     exp.train(setting)



Use CPU
Use Data augmentation method: cost
l2norm True
Mask Rate: 0.5
>>>>>>>start training : tcn-moco_ECL_ftM_sl48_ll48_pl24_dm64_nh8_el2_dl1_df2048_atprob_fc3_ebtimeF_dtTrue_mxTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>


In [10]:
# x = torch.rand(8, 48, 7)

In [11]:
model = exp.train_pretrain(setting)

ECL
ECL
ECL
n_iters 600
	iters: 101 | loss: 5.5239787
	speed: 0.0217s/iter;
	iters: 201 | loss: 5.5154543
	speed: 0.0098s/iter;
	iters: 301 | loss: 5.4937468
	speed: 0.0068s/iter;
	iters: 401 | loss: 5.4851065
	speed: 0.0055s/iter;
	iters: 501 | loss: 5.4821491
	speed: 0.0043s/iter;
	iters: 601 | loss: 5.4777670
	speed: 0.0036s/iter;


In [12]:
from two_forecasting import *
from data.utils import *


In [13]:
eval_forecasting(exp, args)

ECL
ECL
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x torch.Size([256, 201, 321])
x 

({24: {'norm': array([[[[-5.42781055e-02,  4.63351816e-01, -4.08190012e-01, ...,
              1.58715844e-02,  3.18464041e-01,  2.74218321e-02],
            [-2.18893886e-02,  1.77780628e-01, -4.21665370e-01, ...,
              2.93295145e-01,  3.30301106e-01,  2.76744366e-02],
            [-5.12687862e-02, -4.79340553e-04, -4.51956093e-01, ...,
              2.26573110e-01,  3.46425742e-01, -8.69386196e-02],
            ...,
            [-2.65986696e-02,  6.48486733e-01, -3.53113353e-01, ...,
             -3.02216947e-01,  8.59713554e-02, -4.79189157e-02],
            [-4.87088114e-02,  7.37935960e-01, -3.36216837e-01, ...,
             -3.30030441e-01,  1.16492033e-01, -1.10355854e-01],
            [-8.27392638e-02,  6.85320914e-01, -3.60267401e-01, ...,
             -3.40499520e-01,  1.52356625e-01, -1.06175065e-01]],
   
           [[-3.38185698e-01,  5.37991405e-01, -1.10961795e+00, ...,
              1.15418881e-01,  1.05110586e+00,  3.83925200e-01],
            [-2.62049139e-01

In [52]:
print(args.data)
if "ETT" in args.data:
    dataset = "ETT/" + args.data
else:
    dataset = args.data

print(dataset)
if args.features == "M":
    data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols = load_forecast_csv(dataset)
elif args.features == "S":
    data, train_slice, valid_slice, test_slice, scaler, pred_lens, n_covariate_cols = load_forecast_csv(dataset, univar=True)
train_data = data[:, train_slice]

ECL
ECL
ECL


In [30]:
mask=None
encoding_window=None
casual=True
sliding_length=1
sliding_padding=200 
batch_size=256

In [31]:
# batch_size = 16
n_samples, ts_l, _ = data .shape

In [53]:
pred_lens

[24, 48, 168, 336, 720]

In [33]:
org_training = exp.model.training
exp.model.eval()

dataset = TensorDataset(torch.from_numpy(data).to(torch.float))
loader = DataLoader(dataset, batch_size=batch_size)


In [50]:
with torch.no_grad():
    output = []
    for batch in loader:
        x = batch[0]
        if sliding_length is not None:
            reprs = []
            if n_samples < batch_size:
                calc_buffer = []
                calc_buffer_l = 0
            for i in range(0, ts_l, sliding_length):
                l = i - sliding_padding
                r = i + sliding_length + (sliding_padding if not casual else 0)
                x_sliding = torch_pad_nan(
                    x[:, max(l, 0) : min(r, ts_l)],
                    left=-l if l<0 else 0,
                    right=r-ts_l if r>ts_l else 0,
                    dim=1
                )
                if n_samples < batch_size:
                    if calc_buffer_l + n_samples > batch_size:
                        out = _eval_with_pooling(exp.model,
                            torch.cat(calc_buffer, dim=0),
                            mask,
                            slicing=slice(sliding_padding, sliding_padding+sliding_length),
                            encoding_window=encoding_window
                        )
                        reprs += torch.split(out, n_samples)
                        calc_buffer = []
                        calc_buffer_l = 0
                    calc_buffer.append(x_sliding)
                    calc_buffer_l += n_samples
                else:
                    out = self._eval_with_pooling(exp.model,
                        x_sliding,
                        mask,
                        slicing=slice(sliding_padding, sliding_padding+sliding_length),
                        encoding_window=encoding_window
                    )
                    reprs.append(out)

            if n_samples < batch_size:
                if calc_buffer_l > 0:
                    out = _eval_with_pooling(exp.model,
                        torch.cat(calc_buffer, dim=0),
                        mask,
                        slicing=slice(sliding_padding, sliding_padding+sliding_length),
                        encoding_window=encoding_window
                    )
                    reprs += torch.split(out, n_samples)
                    calc_buffer = []
                    calc_buffer_l = 0

            out = torch.cat(reprs, dim=1)
            if encoding_window == 'full_series':
                out = F.max_pool1d(
                    out.transpose(1, 2).contiguous(),
                    kernel_size = out.size(1),
                ).squeeze(1)
        else:
            out = self._eval_with_pooling(x, mask, encoding_window=encoding_window)
            if encoding_window == 'full_series':
                out = out.squeeze(1)

        output.append(out)
        
    output = torch.cat(output, dim=0)

torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
torch.Size([256, 201, 64])
t

In [55]:
features = output.numpy()

In [49]:
def _eval_with_pooling(model, x, mask=None, slicing=None, encoding_window=None):
    out = model(x, mask)[1]
    print(out.shape)
    if encoding_window == 'full_series':
        if slicing is not None:
            out = out[:, slicing]
        out = F.max_pool1d(
            out.transpose(1, 2),
            kernel_size = out.size(1),
        ).transpose(1, 2)

    elif isinstance(encoding_window, int):
        out = F.max_pool1d(
            out.transpose(1, 2),
            kernel_size = encoding_window,
            stride = 1,
            padding = encoding_window // 2
        ).transpose(1, 2)
        if encoding_window % 2 == 0:
            out = out[:, :-1]
        if slicing is not None:
            out = out[:, slicing]

    elif encoding_window == 'multiscale':
        p = 0
        reprs = []
        while (1 << p) + 1 < out.size(1):
            t_out = F.max_pool1d(
                out.transpose(1, 2),
                kernel_size = (1 << (p + 1)) + 1,
                stride = 1,
                padding = 1 << p
            ).transpose(1, 2)
            if slicing is not None:
                t_out = t_out[:, slicing]
            reprs.append(t_out)
            p += 1
        out = torch.cat(reprs, dim=-1)

    else:
        
        if slicing is not None:
            out = out[:, slicing]

    return out.cpu()

In [66]:
def generate_pred_samples(features, data, pred_len, drop=0):
    n = data.shape[1]
    features = features[:, :-pred_len]
    labels = np.stack([ data[:, i:1+n+i-pred_len] for i in range(pred_len)], axis=2)[:, 1:]
    features = features[:, drop:]
    print(features.shape)
    print(labels.shape)
    labels = labels[:, drop:]
    return features.reshape(-1, features.shape[-1]), \
            labels.reshape(-1, labels.shape[2]*labels.shape[3])

In [67]:
train_features, train_labels = generate_pred_samples(features, data, 24, drop=0)

(1, 26280, 64)
(1, 26280, 24, 321)


In [65]:
train_labels.shape

(26280, 7704)

In [40]:
exp.test(setting)

test 2857
test shape: (357, 8, 24, 7) (357, 8, 24, 7)
test shape: (2856, 24, 7) (2856, 24, 7)
mse:0.6696983575820923, mae:0.594174325466156


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(exp.model)

In [ ]:
#TCN
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [ ]:
x = torch.rand(4, 128, 72)

In [ ]:
tcn = TemporalConvNet(128, [512,512], dropout=0.1)

In [ ]:
tcn(x).shape

In [ ]:

lstm = nn.LSTM(128, 512, num_layers=3, batch_first=True, dropout=0.1)

In [ ]:
x = torch.rand(4, 72, 128)
out, (hn, cn) = lstm(x)

In [ ]:
decoder = nn.Linear(512, 7, bias=True)

In [ ]:
decoder(out).shape

In [ ]:
len(out)

In [ ]:
from models.dilated_tcn import DTCN_MoCo, DilatedConvEncoder
import argparse
import os
import torch

In [ ]:
model = DTCN_MoCo(7,
                 64, 
                 7, 
                 e_layer = 2, 
                 pred_leng = 48,
                 freq = 'h',
                 kernel_size = 3,
                 dropout=0.1,
                 K=512,
                 T=1.00,
                 l2norm=True,
                 average_pool = False,
                 data_aug = 'cost'
                 )

In [ ]:
x = torch.rand(8, 48, 7)

In [ ]:
model(x)[1].shape

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

In [ ]:
from torch import nn
x = torch.rand(8, 48, 7)
enc_embedding = nn.Linear(7, 64)
out = enc_embedding(x)

In [ ]:
out.shape

In [ ]:
tcn = DilatedConvEncoder(64, [64] * 2, 3)

In [ ]:
tcn(out.transpose(1, 2)).shape

In [ ]:
tcn.net[0](out.transpose(1, 2))

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from models.weight_norm import WeightNorm
from models.data_aug import Data_Aug, generate_binomial_mask
from typing import Union, Callable, Optional, List
import sys, math, random, copy
from models.embed import TimeFeatureEmbedding
from models.dtcn_encoder import *
import torch.fft as fft
from einops import reduce, rearrange, repeat


class BandedFourierLayer(nn.Module):
    def __init__(self, in_channels, out_channels, band, num_bands, length=201):
        super().__init__()

        self.length = length
        self.total_freqs = (self.length // 2) + 1

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.band = band  # zero indexed
        self.num_bands = num_bands

        self.num_freqs = self.total_freqs // self.num_bands + (self.total_freqs % self.num_bands if self.band == self.num_bands - 1 else 0)

        self.start = self.band * (self.total_freqs // self.num_bands)
        self.end = self.start + self.num_freqs


        # case: from other frequencies
        self.weight = nn.Parameter(torch.empty((self.num_freqs, in_channels, out_channels), dtype=torch.cfloat))
        self.bias = nn.Parameter(torch.empty((self.num_freqs, out_channels), dtype=torch.cfloat))
        self.reset_parameters()

    def forward(self, input):
        # input - b t d
        b, t, _ = input.shape
        input_fft = fft.rfft(input, dim=1)
        output_fft = torch.zeros(b, t // 2 + 1, self.out_channels, device=input.device, dtype=torch.cfloat)
        output_fft[:, self.start:self.end] = self._forward(input_fft)
        return fft.irfft(output_fft, n=input.size(1), dim=1)

    def _forward(self, input):
        output = torch.einsum('bti,tio->bto', input[:, self.start:self.end], self.weight)
        return output + self.bias

    def reset_parameters(self) -> None:
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
        bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
        nn.init.uniform_(self.bias, -bound, bound)

class CoSTEncoder(nn.Module):
    def __init__(self, input_dims, output_dims,
                 kernels: List[int],
                 length: int,
                 hidden_dims=64, depth=10,
                 mask_mode='binomial'):
        super().__init__()

        component_dims = output_dims // 2

        self.input_dims = input_dims
        self.output_dims = output_dims
        self.component_dims = component_dims
        self.hidden_dims = hidden_dims
        self.mask_mode = mask_mode
        self.input_fc = nn.Linear(input_dims, hidden_dims)

        self.feature_extractor = DilatedConvEncoder(
            hidden_dims,
            [hidden_dims] * depth + [output_dims],
            kernel_size=3
        )

        self.repr_dropout = nn.Dropout(p=0.1)

        self.kernels = kernels

        self.tfd = nn.ModuleList(
            [nn.Conv1d(output_dims, component_dims, k, padding=k-1) for k in kernels]
        )

        self.sfd = nn.ModuleList(
            [BandedFourierLayer(output_dims, component_dims, b, 1, length=length) for b in range(1)]
        )

    def forward(self, x, tcn_output=False, mask='all_true'):  # x: B x T x input_dims
        nan_mask = ~x.isnan().any(axis=-1)
        x[~nan_mask] = 0
        x = self.input_fc(x)  # B x T x Ch

        # generate & apply mask
        if mask is None:
            if self.training:
                mask = self.mask_mode
            else:
                mask = 'all_true'

        if mask == 'binomial':
            mask = generate_binomial_mask(x.size(0), x.size(1)).to(x.device)
        elif mask == 'continuous':
            mask = generate_continuous_mask(x.size(0), x.size(1)).to(x.device)
        elif mask == 'all_true':
            mask = x.new_full((x.size(0), x.size(1)), True, dtype=torch.bool)
        elif mask == 'all_false':
            mask = x.new_full((x.size(0), x.size(1)), False, dtype=torch.bool)
        elif mask == 'mask_last':
            mask = x.new_full((x.size(0), x.size(1)), True, dtype=torch.bool)
            mask[:, -1] = False

        mask &= nan_mask
        x[~mask] = 0

        # conv encoder
        x = x.transpose(1, 2)  # B x Ch x T

        print(x.shape)
        x = self.feature_extractor(x)  # B x Co x T

        print(x.shape)

        if tcn_output:
            return x.transpose(1, 2)

        trend = []
        for idx, mod in enumerate(self.tfd):
            print(idx)
            out = mod(x)  # b d t
            if self.kernels[idx] != 1:
                out = out[..., :-(self.kernels[idx] - 1)]

            print(out.shape)
            trend.append(out.transpose(1, 2))  # b t d
        print(len(trend))

        trend = reduce(
            rearrange(trend, 'list b t d -> list b t d'),
            'list b t d -> b t d', 'mean'
        )

        print(x.shape)
        x = x.transpose(1, 2)  # B x T x Co

        print(x.shape)

        season = []
        for mod in self.sfd:
            out = mod(x)  # b t d
            season.append(out)
        season = season[0]

        return trend, self.repr_dropout(season)


class DTCN_MoCo(nn.Module):
    def __init__(self, 
                 input_size, hidden_size, output_size, e_layer, pred_leng, input_length,
                  dropout = 0.1, l2norm = True, data_aug = "cost",
                 device: Optional[str] = 'cuda',
                 K: Optional[int] = 256,
                 m: Optional[float] = 0.999,
                 T: Optional[float] = 1.0):
        super(DTCN_MoCo, self).__init__()
        
        self.K = K
        self.m = m
        self.T = T
        self.device = device
        self.l2norm = l2norm
        self.data_aug = data_aug

        if self.data_aug == "cost":
            print("Use Data augmentation method:", self.data_aug)
            self.data_aug_tool = Data_Aug(sigma=0.5, p=0.5)

        print("l2norm", self.l2norm )

        self.encoder_q = CoSTEncoder(input_size, output_dims = hidden_size, 
                                     kernels = [1, 2, 4, 8, 16, 32, 64],
                                     length = input_length, depth = e_layer)

        self.encoder_k = copy.deepcopy(self.encoder_q)


        # create the encoders
        self.head_q = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )
        self.head_k = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size)
        )

        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data.copy_(param_q.data)  # initialize
            param_k.requires_grad = False  # not update by gradient
        for param_q, param_k in zip(self.head_q.parameters(), self.head_k.parameters()):
            param_k.data.copy_(param_q.data)  # initialize
            param_k.requires_grad = False  # not update by gradient

        self.register_buffer('queue', F.normalize(torch.randn(hidden_size, K), dim=0))
        self.register_buffer('queue_ptr', torch.zeros(1, dtype=torch.long))

        self.decoder = nn.Linear(hidden_size, output_size, bias=True)
        self.drop = nn.Dropout(dropout)
        self.pred_leng = pred_leng
        self.init_weights()

    def init_weights(self):
        self.decoder.bias.data.fill_(0)
        self.decoder.weight.data.normal_(0, 0.01)

    def convert_coeff(self, x, eps=1e-6):
        amp = torch.sqrt((x.real + eps).pow(2) + (x.imag + eps).pow(2))
        phase = torch.atan2(x.imag, x.real + eps)
        return amp, phase

    def instance_contrastive_loss(self, z1, z2):
        B, T = z1.size(0), z1.size(1)
        z = torch.cat([z1, z2], dim=0)  # 2B x T x C
        z = z.transpose(0, 1)  # T x 2B x C
        sim = torch.matmul(z, z.transpose(1, 2))  # T x 2B x 2B
        logits = torch.tril(sim, diagonal=-1)[:, :, :-1]  # T x 2B x (2B-1)
        logits += torch.triu(sim, diagonal=1)[:, :, 1:]
        logits = -F.log_softmax(logits, dim=-1)

        i = torch.arange(B, device=z1.device)
        loss = (logits[:, i, B + i - 1].mean() + logits[:, B + i, i].mean()) / 2
        return loss

    def compute_loss(self, q, k, k_negs):
        
        if self.l2norm:
            q = F.normalize(q, dim=-1)
            k = F.normalize(k, dim=-1)
            k_negs = F.normalize(k_negs, dim=-1)
            
        # compute logits
        # positive logits: Nx1
        l_pos = torch.einsum('nc,nc->n', [q, k]).unsqueeze(-1)
        # negative logits: NxK
        l_neg = torch.einsum('nc,ck->nk', [q, k_negs])

        # logits: Nx(1+K)
        logits = torch.cat([l_pos, l_neg], dim=1)

        # apply temperature
        logits /= self.T

        # labels: positive key indicators - first dim of each batch
        labels = torch.zeros(logits.shape[0], dtype=torch.long).to(q.device)
        loss = F.cross_entropy(logits, labels)

        return loss

    @torch.no_grad()
    def _momentum_update_key_encoder(self):
        """
        Momentum update for key encoder
        """
        for param_q, param_k in zip(self.encoder_q.parameters(), self.encoder_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1 - self.m)
        for param_q, param_k in zip(self.head_q.parameters(), self.head_k.parameters()):
            param_k.data = param_k.data * self.m + param_q.data * (1 - self.m)

    @torch.no_grad()
    def _dequeue_and_enqueue(self, keys):
        batch_size = keys.shape[0]

        ptr = int(self.queue_ptr)
        assert self.K % batch_size == 0

        # replace keys at ptr (dequeue and enqueue)
        self.queue[:, ptr:ptr + batch_size] = keys.T

        ptr = (ptr + batch_size) % self.K
        self.queue_ptr[0] = ptr

    def forward(self, x):
        # compute query features
        x_q = x
        rand_idx = np.random.randint(0, x_q.shape[1])

        print(1)

        q_t, q_s = self.encoder_q(x_q, mask='all_true')
        print(2)
        enc_out = torch.cat([q_t, q_s], dim=-1)

        print(3)

        if q_t is not None:
            q_t = F.normalize(self.head_q(q_t[:, rand_idx]), dim=-1)

        print(4)

        # compute key features
        with torch.no_grad():  # no gradient for keys
            self._momentum_update_key_encoder()  # update key encoder

            if self.data_aug:
                x_k = self.data_aug_tool.cost_transform(x)

                print(5)

            k_t, k_s = self.encoder_k(x_k)
            if k_t is not None:
                k_t = F.normalize(self.head_k(k_t[:, rand_idx]), dim=-1)

        loss = 0

        loss += self.compute_loss(q_t, k_t, self.queue.clone().detach())
        self._dequeue_and_enqueue(k_t)

        q_s = F.normalize(q_s, dim=-1)
        _, k_s = self.encoder_q(x_k)
        k_s = F.normalize(k_s, dim=-1)

        q_s_freq = fft.rfft(q_s, dim=1)
        k_s_freq = fft.rfft(k_s, dim=1)
        q_s_amp, q_s_phase = self.convert_coeff(q_s_freq)
        k_s_amp, k_s_phase = self.convert_coeff(k_s_freq)

        
        seasonal_loss = self.instance_contrastive_loss(q_s_amp, k_s_amp) + \
                        self.instance_contrastive_loss(q_s_phase,k_s_phase)
        loss += (self.alpha * (seasonal_loss/2))

        y = self.decoder(enc_out)
        
        return y[:,-self.pred_leng:,:], enc_out, loss


In [ ]:
net = DTCN_MoCo(7, 320,
                 7, 10, 96, 96, 0.1,  device = "cpu")

In [ ]:
out_t, out_s = net(torch.rand(8, 96, 7))

In [ ]:
out_t.shape

In [ ]:
out_s.shape

In [ ]:
out = torch.cat([out_t[:, -1], out_s[:, -1]], dim=-1)

In [ ]:
out.shape

In [69]:
!squeue -u chiyuzh


             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
            732635      main      jpt  chiyuzh  R    4:55:42      1 compute06
